In [1]:
from dash import Dash, html, dcc, Input, Output  # pip install dash
import plotly.express as px
import dash_ag_grid as dag                       # pip install dash-ag-grid
import dash_bootstrap_components as dbc          # pip install dash-bootstrap-components
import pandas as pd                              # pip install pandas

import matplotlib                                # pip install matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import base64
from io import BytesIO

import requests
import numpy as np

import plotly.graph_objects as go

In [2]:
df2 = pd.read_csv("famiglie.csv", quotechar="'")

selected_columns = ['Territorio', 'COICOP', 'Coicop (DESC)', 'Anno', 'Spesa media', 'Tipologia famigliare']
df2 = df2[selected_columns]

#spesa media
df2['Spesa media'] = pd.to_numeric(df2['Spesa media'], errors='coerce')
df2['Spesa media'] = df2['Spesa media'].replace(np.nan, -1, regex=True)
df2['Spesa media'] = df2['Spesa media'].astype(float)
df2 = df2[df2['Spesa media'] >= 0]


#categorie

#root
df2.loc[df2['COICOP'] == 'NON_FOOD', 'Root'] = 'Non alimentari'
df2.loc[((df2['COICOP'].str.len() == 2)) | (df2['COICOP'] == 'ALL'), 'Root'] = df2['Coicop (DESC)']

##first level
df2.loc[((df2['COICOP'].str.len() == 4) & (df2['Root'].isna())), 'First_level'] = df2['Coicop (DESC)']

#fill the gaps
mask = df2['Root'].isna()
df2['Root'] = df2['Root'].fillna(method='ffill')

macro_category_family=df2[df2['First_level'].isna()]['Coicop (DESC)'].unique()

In [3]:
display(df2)

,Territorio,COICOP,Coicop (DESC),Anno,Spesa media,Tipologia famigliare,Root,First_level
0,Italia,01,-- prodotti alimentari e bevande analcoliche,1997,390.66,Totale famiglie,-- prodotti alimentari e bevande analcoliche,NaN
1,Italia,01,-- prodotti alimentari e bevande analcoliche,1998,390.34,Totale famiglie,-- prodotti alimentari e bevande analcoliche,NaN
2,Italia,01,-- prodotti alimentari e bevande analcoliche,1999,385.92,Totale famiglie,-- prodotti alimentari e bevande analcoliche,NaN
3,Italia,01,-- prodotti alimentari e bevande analcoliche,2000,392.97,Totale famiglie,-- prodotti alimentari e bevande analcoliche,NaN
4,Italia,01,-- prodotti alimentari e bevande analcoliche,2001,399.99,Totale famiglie,-- prodotti alimentari e bevande analcoliche,NaN
...,...,...,...,...,...,...,...,...
13047,Mezzogiorno,ALL,Totale,2017,2270.45,Altre tipologie,Totale,NaN
13048,Mezzogiorno,ALL,Totale,2018,2199.09,Altre tipologie,Totale,NaN
13049,Mezzogiorno,ALL,Totale,2019,2305.80,Altre tipologie,Totale,NaN
13050,Mezzogiorno,ALL,Totale,2020,2212.50,Altre tipologie,Totale,NaN


In [13]:
mask = df2[(df2['Anno'] == 2020) & (df2['Territorio']=='Italia') & (df2['Coicop (DESC)']=='Totale')]
fig = px.bar_polar(
    mask,
    r=mask["Spesa media"],
    theta=mask["Tipologia famigliare"],
    #color=mask["total exports"],
    template="plotly_white",
    color_continuous_scale=px.colors.sequential.Plasma,
    log_r=False,
)
fig.show()

In [15]:
mask = df2[ (df2['Coicop (DESC)']=='Totale') & (df2['Tipologia famigliare']=='Totale famiglie')]

fig = px.line(
    mask,
    x="Anno",
    y="Spesa media",
    color="Territorio",
    markers=True,
    title="Life Expectancy for Countries with High Population",
)
fig.update(layout=dict(title=dict(x=0.5)))

In [20]:
mask = df2[ (df2['Coicop (DESC)']=='Totale') ]

fig = px.bar(
    mask,
    x="Tipologia famigliare", 
    y="Spesa media", 
    color="Territorio", 
    title="Long-Form Input",
    animation_frame="Anno",
)
fig.show()

In [17]:
#mask = df2[ (df2['Coicop (DESC)']=='Totale') & (df2['Anno']==2014)]

mask = df2[(df2['Territorio']=='Italia') & (df2['Coicop (DESC)'].isin(macro_category_family)) & (~df2['Coicop (DESC)'].isin(['Totale']))]

fig=px.scatter(
            mask,
            x="Tipologia famigliare",
            y="Coicop (DESC)",
            size="Spesa media",
            hover_name="Territorio",
            #animation_frame="Anno",
        )
fig.update_layout(
    xaxis={'type': 'category'},  # Ensure x-axis is treated as category
    yaxis={'type': 'category'}   # Ensure y-axis is treated as category
)

fig.show()

In [15]:
def normalize_rows(df_in):
    normalized_df = df_in.copy()
    normalized_df['Spesa media nn'] = normalized_df['Spesa media']
    max_values = df_in.groupby('Coicop (DESC)')['Spesa media'].transform('max')
    min_values = df_in.groupby('Coicop (DESC)')['Spesa media'].transform('min')
    normalized_df['Spesa media'] = (df_in['Spesa media'] - min_values) / (max_values - min_values)
    return normalized_df

In [40]:
n_df2=normalize_rows(df2)
mask = n_df2[(df2['Territorio']=='Italia') &
                (df2['Coicop (DESC)'].isin(macro_category_family)) & 
                (~df2['Coicop (DESC)'].isin(['Totale', 'Non alimentari']))].groupby(['Anno', 'Coicop (DESC)', 'Tipologia famigliare']).mean('Spesa media')

heatmap_data = mask.pivot_table(
    index='Coicop (DESC)', 
    columns='Tipologia famigliare', 
    values='Spesa media', 
    aggfunc='mean'  # You can change the aggregation function as needed
).fillna(0)  # Fill NaN values with 0 for better heatmap visualization

# Now use Plotly Express to create the heatmap
fig = px.imshow(
    heatmap_data,
    labels=dict(
        x="Tipologia famigliare", 
        y="Categoria di spesa", 
        color="Spesa media"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
)

fig.update_layout(
    autosize=False,
    width=1200,  # Set the width as desired
    height=800,   # Set the height as desired,
    annotations=[]
)

fig.update_traces(hovertemplate='Spesa media: %{customdata}€', customdata=mask['Spesa media nn'])
fig.show()


In [53]:
n_df2=normalize_rows(df2)
mask = n_df2[(df2['Territorio']=='Italia') &
                (df2['Coicop (DESC)'].isin(macro_category_family)) & 
                (~df2['Coicop (DESC)'].isin(['Totale', 'Non alimentari']))].groupby(['Anno', 'Coicop (DESC)', 'Tipologia famigliare']).mean('Spesa media')

heatmap_data = mask.pivot_table(
    index='Coicop (DESC)', 
    columns='Tipologia famigliare', 
    values='Spesa media', 
    aggfunc='mean'
).fillna(0)

heatmap_data_nn = mask.pivot_table(
    index='Coicop (DESC)', 
    columns='Tipologia famigliare', 
    values='Spesa media nn', 
    aggfunc='mean'
).fillna(0)

text = [[f"Categoria di spesa: {index}<br>Tipologia famigliare: {col}<br>Spesa media: {heatmap_data_nn.loc[index, col]:.2f}"
         for col in heatmap_data.columns] for index in heatmap_data.index]

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    hoverongaps=False,
    text=text,
    hoverinfo='text',
    colorscale="burgyl",
))

fig.update_layout(width=1200, height=800, coloraxis_showscale=False)
fig.update(layout_coloraxis_showscale=False)
fig.update_coloraxes(showscale=False, colorbar=None)

fig.show()